In [54]:
import tensorflow as tf
from hailo_sdk_client import ClientRunner

import os
import json
import numpy as np
from PIL import Image

from tensorflow.python.eager.context import eager_mode
def preproc(image, output_height=224, output_width=224, resize_side=256):
    ''' imagenet-standard: aspect-preserving resize to 256px smaller-side, then central-crop to 224px'''
    with eager_mode():
        h, w = image.shape[0], image.shape[1]
        scale = tf.cond(tf.less(h, w), lambda: resize_side / h, lambda: resize_side / w)
        resized_image = tf.compat.v1.image.resize_bilinear(tf.expand_dims(image, 0), [int(h*scale), int(w*scale)])
        cropped_image = tf.compat.v1.image.resize_with_crop_or_pad(resized_image, output_height, output_width)
        return tf.squeeze(cropped_image)


class HDCResnetTF(): 
    #def resnet18_tf2hef( harch='hailo8', dstfn='resnet18.hef'):
    def __init__( self, harch):
        model_name = 'resnet_v1_18'
        ckpt_path = '../models/resnet_v1_18.ckpt'

        start_node = 'resnet_v1_18/conv1/Pad'
        end_node = 'resnet_v1_18/predictions/Softmax'

        runner = ClientRunner(hw_arch=harch)
        hn, npz = runner.translate_tf_model(ckpt_path, model_name, start_node_names=[start_node], end_node_names=[end_node])
        #hailo_model_har_name = '{}_h8l_model.har'.format(model_name)
        #runner.save_har(hailo_model_har_name)
        self.runner = runner
    
    
    def load_dataset(self):
        images_path = '../data'
        images_list = [img_name for img_name in os.listdir(images_path) if
                       os.path.splitext(img_name)[1] == '.jpg']
        calib_dataset = np.zeros((len(images_list), 224, 224, 3), dtype=np.float32)
        for idx, img_name in enumerate(sorted(images_list)):
            img = np.array(Image.open(os.path.join(images_path, img_name)))
            img_preproc = preproc(img)
            calib_dataset[idx,:,:,:] = img_preproc.numpy().astype(np.uint8)
        self.calib_dataset = calib_dataset
        
    def optimize(self):
        csv_path = 'translated_params_resnet_v1_18.csv'
        alls_lines = [
            # Add normalization layer with mean [123.675, 116.28, 103.53] and std [58.395, 57.12, 57.375])
            'normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n',
            # For multiple input nodes:
            # {normalization_layer_name_1} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_1_from_hn})\n', 
            # {normalization_layer_name_2} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_2_from_hn})\n', 
            # ...
            'model_optimization_config(calibration, batch_size=2)\n',  # Batch size is 8 by default
        ]

        # Save the commands in an .alls file, this is the Model Script
        open('simple_script.alls','w').writelines(alls_lines)

        # Load the model script to ClientRunner so it will be considered on optimization
        self.runner.load_model_script('simple_script.alls')

        # For a single input layer, could use the shorter version - just pass the dataset to the function
        # runner.optimize(calib_dataset)
        # For multiple input nodes, the calibration dataset could also be a dictionary with the format:
        # {input_layer_name_1_from_hn: layer_1_calib_dataset, input_layer_name_2_from_hn: layer_2_calib_dataset}
        hn_layers = self.runner.get_hn_dict()['layers']
        print([layer for layer in hn_layers if hn_layers[layer]['type'] == 'input_layer']) # See available input layer names
        calib_dataset_dict = {'resnet_v1_18/input_layer1': self.calib_dataset} # In our case there is only one input layer
        self.runner.optimize(calib_dataset_dict)
        
    def build(self, dstfn):
        self.load_dataset()
        self.optimize()
        hef = self.runner.compile()
        with open(dstfn, 'wb') as f:
            f.write(hef)
        
        
h8s1 = HDCResnetTF( 'hailo8')
h8s1.build( 'resnet.hef')

h8s2 = HDCResnetTF( 'hailo8l')
h8s2.build( 'resnet_h8l.hef')



[warning] This model has Batch normalization layers exported from a training graph, expect different inference results between native emulation and original TF session.
[info] Translation completed on TensorFlow 1.x model resnet_v1_18
[info] Initialized runner for resnet_v1_18
[info] Loading model script on resnet_v1_18
['resnet_v1_18/input_layer1']
[info] Adding normalization layers
[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[info] Using calibration set of 64 entries
[info] Starting Stats Collector


Calibration: 100%|█████████████████████████████████████████| 64/64 [00:07<00:00,  8.70entries/s]

[info] Stats Collector is done (completion time is 00:00:07.39)


[info] Bias Correction skipped
[info] Adaround skipped
[info] Fine Tune skipped
[info] Model Optimization is done
after optimize
[info] Loading network parameters
[info] Starting Hailo allocation and compilation flow
[info] Using Single-context flow
[info] Resources optimization guidelines: Strategy -> GREEDY Objective -> MAX_FPS
[info] Resources optimization params: max_control_utilization=75%, max_compute_utilization=75%, max_memory_utilization (weights)=75%, max_input_aligner_utilization=75%, max_apu_utilization=75%
[info] Starting context partition
[info] Context partition is done (0s 1ms)
[info] Adding format conversion layer 'auto_reshape_from_input_layer1_to_normalization1' after input_layer1
[info] output_layer1: Pass
[info] input_layer1: Pass
[info] maxpool1_fs: Pass
[info] conv1_sdc: Pass
[info] normalization1: Pass
[info] maxpool1_d1: Pass
[info] smuffers_shortcut_conv3_to_conv4: Pass
[info] maxpool1_d0: Pass
[info] maxpool1_dc: Pass
[info] maxpool1_d2: Pass
[info] maxpool1_

Calibration: 100%|█████████████████████████████████████████| 64/64 [00:07<00:00,  8.35entries/s]

[info] Stats Collector is done (completion time is 00:00:07.69)


[info] Bias Correction skipped
[info] Adaround skipped
[info] Fine Tune skipped
[info] Model Optimization is done
after optimize
[info] Loading network parameters
[info] Starting Hailo allocation and compilation flow
[info] Using Multi-context flow
[info] Resources optimization guidelines: Strategy -> GREEDY Objective -> MAX_FPS
[info] Resources optimization params: max_control_utilization=75%, max_compute_utilization=75%, max_memory_utilization (weights)=75%, max_input_aligner_utilization=75%, max_apu_utilization=75%
[info] Starting context partition
[info] Running Context Partitioner: mode=enabled, max_utilization=100%, max_control_utilization=30%, max_compute_utilization=30%, max_memory_utilization=30%
goal_network_control_utilization=100%, goal_network_compute_utilization=100%, goal_network_memory_utilization=100%, goal_network_weights_utilization=100%
[info] auto_context_0:
normalization1, conv1, maxpool1, conv2, conv3, conv4, conv5, conv6, conv7, conv8, conv9, conv10, conv11, con

In [55]:
!ls -al resnet.hef resnet_h8l.hef

-rw-r--r-- 1 root root 28902163 Mar 28 11:25 resnet_h8l.hef
-rw-r--r-- 1 root root 11423311 Mar 28 11:22 resnet.hef


In [56]:
!hailortcli scan
#!hailortcli benchmark -h
!hailortcli benchmark  --no-power true resnet.hef 
!hailortcli benchmark  --no-power true resnet_h8l.hef 


Hailo Devices:
[-] Device: 0000:05:00.0
[-] Device: 0000:0e:00.0
Starting Measurements...
Measuring FPS in hw_only mode
Network resnet_v1_18/resnet_v1_18: 6% | 2337 | FPS: 2336.07 | ETA: 00:00:14
Network resnet_v1_18/resnet_v1_18: 13% | 4679 | FPS: 2337.62 | ETA: 00:00:13
Network resnet_v1_18/resnet_v1_18: 20% | 7022 | FPS: 2338.56 | ETA: 00:00:12
Network resnet_v1_18/resnet_v1_18: 26% | 9364 | FPS: 2338.79 | ETA: 00:00:11
Network resnet_v1_18/resnet_v1_18: 33% | 11704 | FPS: 2338.77 | ETA: 00:00:10
Network resnet_v1_18/resnet_v1_18: 40% | 14046 | FPS: 2338.88 | ETA: 00:00:09
Network resnet_v1_18/resnet_v1_18: 46% | 16389 | FPS: 2339.11 | ETA: 00:00:08
Network resnet_v1_18/resnet_v1_18: 53% | 18731 | FPS: 2339.15 | ETA: 00:00:07
Network resnet_v1_18/resnet_v1_18: 60% | 21072 | FPS: 2339.17 | ETA: 00:00:06
Network resnet_v1_18/resnet_v1_18: 66% | 23414 | FPS: 2339.19 | ETA: 00:00:05
Network resnet_v1_18/resnet_v1_18: 73% | 25756 | FPS: 2339.22 | ETA: 00:00:04
Network resnet_v1_18/resnet

In [58]:
!hailortcli scan
#!hailortcli benchmark  --no-power true --device-id=0000:0e:00.0 


Hailo Devices:
[-] Device: 0000:05:00.0
[-] Device: 0000:0e:00.0


In [57]:
!ls -al *.hef
!hailortctl benchmark --no-power true resnet.hef
!hailortctl benchmark --no-power true resnet_h8l.hef

-rw-r--r-- 1 root root 28902163 Mar 28 11:25 resnet_h8l.hef
-rw-r--r-- 1 root root 11423311 Mar 28 11:22 resnet.hef
-rw-r--r-- 1 root root 28900745 Mar 28 09:44 resnet_v1_18_h8l.hef
-rw-r--r-- 1 root root 11423561 Mar 28 07:12 resnet_v1_18.hef
(hailo) Running command 'benchmark' with 'hailortcli'
Starting Measurements...
Measuring FPS in hw_only mode
Network resnet_v1_18/resnet_v1_18: 6% | 2339 | FPS: 2336.81 | ETA: 00:00:14
Network resnet_v1_18/resnet_v1_18: 13% | 4681 | FPS: 2338.01 | ETA: 00:00:13
Network resnet_v1_18/resnet_v1_18: 20% | 7022 | FPS: 2338.46 | ETA: 00:00:12
Network resnet_v1_18/resnet_v1_18: 26% | 9363 | FPS: 2338.68 | ETA: 00:00:11
Network resnet_v1_18/resnet_v1_18: 33% | 11705 | FPS: 2338.83 | ETA: 00:00:10
Network resnet_v1_18/resnet_v1_18: 40% | 14048 | FPS: 2339.10 | ETA: 00:00:09
Network resnet_v1_18/resnet_v1_18: 46% | 16389 | FPS: 2339.02 | ETA: 00:00:08
Network resnet_v1_18/resnet_v1_18: 53% | 18731 | FPS: 2339.08 | ETA: 00:00:07
Network resnet_v1_18/resnet_